<a href="https://colab.research.google.com/github/100jy/voice_competition/blob/master/k_fold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 드라이브 연동 및 모듈 임포트

In [1]:
#구글 드라이브 연동
# 클라우드 권한 획득
from google.colab import auth, drive
auth.authenticate_user()
drive.mount('/content/drive')
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
import tensorflow as tf
import keras
from tqdm import tqdm
from glob import glob
from scipy.io import wavfile
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, Callback
from tensorflow.keras.optimizers import *
from tensorflow.keras import regularizers
from tensorflow.keras.regularizers import l2
import tensorflow_addons as tfa

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# 데이터 로드 및 TPU 초기화

In [11]:
def set_batch(size=1600,seed=100,dim=2): 
  GCS_PATH = 'gs://data_bucket_9586/' 
  AUTO = tf.data.experimental.AUTOTUNE # used in tf.data.Dataset API

  if dim == 2:
    def read_train(example):
      feature = {'data': tf.io.VarLenFeature(dtype = tf.float32),
                'label': tf.io.VarLenFeature(dtype = tf.float32),
                }
      example = tf.io.parse_single_example(example, feature)

      data = tf.sparse.to_dense(example['data'])
      label = tf.sparse.to_dense(example['label'])

      data = tf.reshape(data,(299,299,1))
      label = tf.reshape(label,(30,1))

      return data, label

    filenames = tf.io.gfile.glob(GCS_PATH + 'train_resize_*.tfrec')
    #반만 보기...
    spit_len = int(len(filenames) * 0.9)

    np.random.seed(seed)
    np.random.shuffle(filenames)

    train_dataset = tf.data.TFRecordDataset(filenames[:spit_len],num_parallel_reads=AUTO)
    #batch_size = 3200
    train_dataset = train_dataset.map(read_train, num_parallel_calls=AUTO).batch(size).prefetch(1)

    val_dataset = tf.data.TFRecordDataset(filenames[spit_len:spit_len+1],num_parallel_reads=AUTO)
    #batch_size = 3200
    val_dataset = val_dataset.map(read_train, num_parallel_calls=AUTO).batch(size).prefetch(1)  

    for record in train_dataset.take(1):
      print(len(record))
      input_shape = (record[0].shape[1],record[0].shape[2],record[0].shape[3])
      output_shape = record[1].shape[1]

    print(input_shape)
    print(output_shape)
  
  else:
    GCS_PATH = 'gs://data_bucket_9586/' 
    AUTO = tf.data.experimental.AUTOTUNE # used in tf.data.Dataset API

    def read_train(example):
      feature = {'data': tf.io.VarLenFeature(dtype = tf.float32),
                'label': tf.io.VarLenFeature(dtype = tf.float32),
                }
      example = tf.io.parse_single_example(example, feature)

      data = tf.sparse.to_dense(example['data'])
      label = tf.sparse.to_dense(example['label'])

      data = tf.reshape(data,(11025,1))
      label = tf.reshape(label,(30,1))

      return data, label

    filenames = tf.io.gfile.glob(GCS_PATH + 'train_wave_*.tfrec')
    spit_len = int(len(filenames) * 0.8)

    np.random.seed(seed)
    np.random.shuffle(filenames)

    train_dataset = tf.data.TFRecordDataset(filenames[:spit_len],num_parallel_reads=AUTO)
    #batch_size = 3200
    train_dataset = train_dataset.map(read_train, num_parallel_calls=AUTO).batch(size).prefetch(1)

    val_dataset = tf.data.TFRecordDataset(filenames[spit_len:],num_parallel_reads=AUTO)
    #batch_size = 3200
    val_dataset = val_dataset.map(read_train, num_parallel_calls=AUTO).batch(size).prefetch(1)  

    for record in train_dataset.take(1):
      print(len(record))
      input_shape = (record[0].shape[1])
      output_shape = record[1].shape[1]

    print(input_shape)
    print(output_shape)

  # Detect hardware
  try:
    tpu_resolver = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
  except ValueError:
    tpu_resolver = None
    gpus = tf.config.experimental.list_logical_devices("GPU")

  # Select appropriate distribution strategy
  if tpu_resolver:
    tf.config.experimental_connect_to_cluster(tpu_resolver)
    tf.tpu.experimental.initialize_tpu_system(tpu_resolver)
    strategy = tf.distribute.experimental.TPUStrategy(tpu_resolver)

  elif len(gpus) > 1:
    strategy = tf.distribute.MirroredStrategy([gpu.name for gpu in gpus])

  elif len(gpus) == 1:
    strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU

  else:
    strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU

  return strategy, train_dataset, val_dataset

# 모델 정의

# 1D CNN

In [3]:
def conv1d_bn(x, filters, kernel_size, padding='same', strides=1, activation='relu'):
    x = Conv1D(filters, kernel_size, kernel_initializer='he_normal', padding=padding, strides=strides)(x)
    x = BatchNormalization()(x)
    if activation:
        x = Activation(activation)(x)
    
    return x

def filter_module(x,filters,weight_decay=1e-4):
    x_1 = conv1d_bn(x,filters=filters, kernel_size=3, padding='same')
    x_1 = MaxPooling1D(3, padding = "same", strides = 3)(x_1)

    x_3 = conv1d_bn(x,filters=filters, kernel_size=5, padding='same')
    x_3 = MaxPooling1D(3, padding = "same", strides = 3)(x_3)

    x_4 = conv1d_bn(x,filters=filters, kernel_size=1, padding='same')
    x_4 = MaxPooling1D(3, padding = "same", strides = 3)(x_4)
   
    return concatenate([x_1,x_3,x_4])

def wave_model(input_shape):
  inputs = Input(input_shape)
  x = BatchNormalization(axis=-1, momentum=0.99, epsilon=1e-3, center=True, scale=True)(inputs)
  #First Conv1D layer
  x = Conv1D(8,13, padding='valid', activation='relu', strides=1)(x)
  x = MaxPooling1D(3)(x)
  x = Dropout(0.3)(x)
  #Second Conv1D layer
  x = Conv1D(16, 11, padding='valid', activation='relu', strides=1)(x)
  x = MaxPooling1D(3)(x)
  x = Dropout(0.3)(x)
  #Third Conv1D layer
  x = Conv1D(32, 9, padding='valid', activation='relu', strides=1)(x)
  x = MaxPooling1D(3)(x)
  x = Dropout(0.3)(x)
  x = BatchNormalization(axis=-1, momentum=0.99, epsilon=1e-3, center=True, scale=True)(x)
  x = Bidirectional(GRU(128, return_sequences=True), merge_mode='sum')(x)
  x = Bidirectional(GRU(128, return_sequences=True), merge_mode='sum')(x)
  x = Bidirectional(GRU(128, return_sequences=False), merge_mode='sum')(x)
  x = BatchNormalization(axis=-1, momentum=0.99, epsilon=1e-3, center=True, scale=True)(x)
  #Flatten layer
  # x = Flatten()(x)
  #Dense Layer 1
  x = Dense(256, activation='relu')(x)
  outputs = Dense(30, activation="softmax")(x)

  return Model(inputs,outputs)

## InceptionNet like CNN

In [4]:
def inception_model_1(input_shape):
  def inception_module(x, o_1=64,r_3 =64, o_3 =128,r_5=16,o_5=32,pool=32):
    #size_1 filter
    x_1 = Conv2D(o_1,1,padding='same')(x)

    #size_1 + size_5 filter
    x_3 = Conv2D(r_5,1,padding='same')(x)
    x_3 = Conv2D(o_5,5,padding='same')(x_3)

    #pooling
    x_4 = MaxPooling2D(pool_size = (3,3),strides =1,padding='same')(x)
    x_4 = Conv2D(pool, 1, padding='same')(x_4)

    return concatenate([x_1,x_3,x_4])



  inp = Input(input_shape) 

  x = Conv2D(64, (7, 7), strides = 2, padding = "same")(inp)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = MaxPooling2D((3, 3), padding = "same", strides = 2)(x)
  x = Conv2D(64, (1, 1), strides = 1, padding = "same")(x)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = Conv2D(192, (3, 3), strides = 1, padding = "same")(x)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = MaxPooling2D((3, 3), padding = "same", strides = 2)(x)

  x = inception_module(x, o_1=64, r_3=64, o_3=128, r_5=16, o_5=32, pool=32)
  x = BatchNormalization()(x)
  x = inception_module(x, o_1=128, r_3=128, o_3=192, r_5=32, o_5=96, pool=64)
  x = BatchNormalization()(x)
  x = MaxPooling2D(pool_size=(3, 3), strides=2, padding='same')(x)
  x = inception_module(x, o_1=192, r_3=96, o_3=208, r_5=16, o_5=48, pool=64)
  x = BatchNormalization()(x)
  
  x = inception_module(x, o_1=160, r_3=112, o_3=224, r_5=24, o_5=64, pool=64)
  x = BatchNormalization()(x)
  x = inception_module(x, o_1=128, r_3=128, o_3=256, r_5=24, o_5=64, pool=64)
  x = BatchNormalization()(x)
  x = inception_module(x, o_1=112, r_3=144, o_3=288, r_5=32, o_5=64, pool=64)
  x = BatchNormalization()(x)
  x = inception_module(x, o_1=256, r_3=160, o_3=320, r_5=32, o_5=128, pool=128)
  x = BatchNormalization()(x)

  x = MaxPooling2D(pool_size=(2, 2), strides=1, padding='same')(x)
  x = inception_module(x, o_1=256, r_3=160, o_3=320, r_5=32, o_5=128, pool=128)
  x = BatchNormalization()(x)
  x = inception_module(x, o_1=384, r_3=192, o_3=384, r_5=48, o_5=128, pool=128)
  x = BatchNormalization()(x)
  

  x = AveragePooling2D(pool_size=(2, 2), strides=3)(x)
  x = Conv2D(128, (1, 1),padding = "same")(x)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = Flatten()(x)
  x = Dropout(0.5)(x)

  output = Dense(30, activation = "softmax")(x)

  return Model(inp,output)


## SE_ResNet

In [5]:
import tensorflow.keras.backend as K

def conv2d_bn(x, filters, kernel_size, padding='same', strides=1, activation='relu'):
    x = Conv2D(filters, kernel_size, kernel_initializer='he_normal', padding=padding, strides=strides)(x)
    x = BatchNormalization()(x)
    if activation:
        x = Activation(activation)(x)
    
    return x


def SE_block(input_tensor, reduction_ratio=16):
    ch_input = K.int_shape(input_tensor)[-1]
    ch_reduced = ch_input//reduction_ratio
    
    # Squeeze
    x = GlobalAveragePooling2D()(input_tensor) # Eqn.2
    
    # Excitation
    x = Dense(ch_reduced, kernel_initializer='he_normal', activation='relu', use_bias=False)(x) # Eqn.3
    x = Dense(ch_input, kernel_initializer='he_normal', activation='sigmoid', use_bias=False)(x) # Eqn.3
    
    x = Reshape( (1, 1, ch_input) )(x)
    x = Multiply()([input_tensor, x]) # Eqn.4
    
    return x
   

def SE_residual_block(input_tensor, filter_sizes, strides=1, reduction_ratio=16):
    filter_1, filter_2, filter_3 = filter_sizes
    
    x = conv2d_bn(input_tensor, filter_1, (1, 1), strides=strides)
    x = conv2d_bn(x, filter_2, (3, 3))
    x = conv2d_bn(x, filter_3, (1, 1), activation=None)
    
    x = SE_block(x, reduction_ratio)
    
    projected_input = conv2d_bn(input_tensor, filter_3, (1, 1), strides=strides, activation=None) if K.int_shape(input_tensor)[-1] != filter_3 else input_tensor
    shortcut = Add()([projected_input, x])
    shortcut = Activation(activation='relu')(shortcut)
    
    return shortcut
 

def stage_block(input_tensor, filter_sizes, blocks, reduction_ratio=16, stage=''):
    strides = 2 if stage != '2' else 1
    
    x = SE_residual_block(input_tensor, filter_sizes, strides, reduction_ratio) # projection layer

    for i in range(blocks-1):
        x = SE_residual_block(x, filter_sizes, reduction_ratio=reduction_ratio)
    
    return x
    

def SE_ResNet50(input_shape, classes=30):
    model_input = Input(input_shape)
    stage_1 = conv2d_bn(model_input, 64, (7, 7), strides=2, padding='same') # (112, 112, 64)
    stage_1 = MaxPooling2D((3, 3), strides=2, padding='same')(stage_1) # (56, 56, 64)
    
    stage_2 = stage_block(stage_1, [64, 64, 256], 3, reduction_ratio=16, stage='2')
    stage_3 = stage_block(stage_2, [128, 128, 512], 4, reduction_ratio=16, stage='3') # (28, 28, 512)
    stage_4 = stage_block(stage_3, [256, 256, 1024], 6, reduction_ratio=16, stage='4') # (14, 14, 1024)
    stage_5 = stage_block(stage_4, [512, 512, 2048], 3, reduction_ratio=16, stage='5') # (7, 7, 2048)

    gap = GlobalAveragePooling2D()(stage_5)
    x = Dropout(0.8)(gap)
    
    model_output = Dense(classes, activation='softmax', kernel_initializer='he_normal')(x) # 'softmax'
    
    model = Model(inputs=model_input, outputs=model_output, name='SE-ResNet50')
        
    return model

## DenseNet


In [6]:
def Conv_Block(x, growth_rate, activation='relu'):
    x_l = BatchNormalization()(x)
    x_l = Activation(activation)(x_l)
    x_l = Conv2D(growth_rate*4, (1, 1), padding='same', kernel_initializer='he_normal')(x_l)
    
    x_l = BatchNormalization()(x_l)
    x_l = Activation(activation)(x_l)
    x_l = Conv2D(growth_rate, (3, 3), padding='same', kernel_initializer='he_normal')(x_l)
    
    x = Concatenate()([x, x_l])
    
    return x

def Dense_Block(x, layers, growth_rate=32):
    for i in range(layers):
        x = Conv_Block(x, growth_rate)
    return x

def Transition_Layer(x, compression_factor=0.5, activation='relu'):
    reduced_filters = int(K.int_shape(x)[-1] * compression_factor)
    
    x = BatchNormalization()(x)
    x = Activation(activation)(x)
    x = Conv2D(reduced_filters, (1, 1), padding='same', kernel_initializer='he_normal')(x)
    
    x = AveragePooling2D((2, 2), padding='same', strides=2)(x)
    
    return x

layers_in_block = {'DenseNet-121' : [6, 12, 24, 16],
                   'DenseNet-169' : [6, 12, 32, 32],
                   'DenseNet-201' : [6, 12, 48, 32],
                   'DenseNet-265' : [6, 12, 64, 48]}

base_growth_rate = 32

def DenseNet(model_input, classes, densenet_type='DenseNet-121'):
    model_input = Input(model_input)
    x = Conv2D(base_growth_rate*2, (7, 7), padding='same', strides=2, kernel_initializer='he_normal')(model_input) # (224, 224, 3) -> (112, 112, 64)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = MaxPooling2D((3, 3), padding='same', strides=2)(x) # (112, 112, 64) -> (56, 56, 64)
    
    x = Dense_Block(x, layers_in_block[densenet_type][0], base_growth_rate)
    x = Transition_Layer(x, compression_factor=0.5)
    x = Dense_Block(x, layers_in_block[densenet_type][1], base_growth_rate)
    x = Transition_Layer(x, compression_factor=0.5)
    x = Dense_Block(x, layers_in_block[densenet_type][2], base_growth_rate)
    x = Transition_Layer(x, compression_factor=0.5)
    x = Dense_Block(x, layers_in_block[densenet_type][3], base_growth_rate)
    
    x = GlobalAveragePooling2D()(x)
    
    model_output = Dense(classes, activation='softmax', kernel_initializer='he_normal')(x)

    model = Model(model_input, model_output, name=densenet_type)
    
    return model

# 필요 함수 정의

In [7]:
def fit_model(id, n_epoch,strategy, train_dataset, val_dataset, road_weight =False,model_name ='m1',optimizer_set='radam'):
  model_path = 'drive/My Drive/data/model_{}_{}/'.format(model_name,id)
 
  if not os.path.exists(model_path):
    os.mkdir(model_path)
      
  # validattion 기준 모델 갱신
  model_file = model_path + 'epoch_{epoch:03d}_val_{val_loss:3f}.hdf5'
  checkpoint = ModelCheckpoint(filepath = model_file, monitor = 'val_loss', verbose = 1, save_best_only =True)

  if optimizer_set == 'radam':
    radam = tfa.optimizers.RectifiedAdam()
    optimizer  = tfa.optimizers.Lookahead(radam, sync_period=3, slow_step_size=0.5)

  elif optimizer_set == 'lamb':
    ramb = tfa.optimizers.LAMB()
    optimizer  = tfa.optimizers.Lookahead(ramb, sync_period=3, slow_step_size=0.5)


  with strategy.scope():
    if model_name == 'inception_1':
      model = inception_model_1((299,299,1))
    elif model_name == 'SE_ResNet':
      model = SE_ResNet50((299,299,1),30)
    elif model_name == 'DenseNet':
      model = DenseNet((299,299,1),30)
    elif model_name == '1D_model':
      model = wave_model((11025,1))
    model.compile(loss=tf.keras.losses.KLDivergence(), optimizer = optimizer)
    if road_weight:
      model.load_weights(glob('drive/My Drive/data/model_high_resol_{}/*.hdf5'.format(id))[-1])
    history = model.fit(train_dataset, epochs = n_epoch, validation_data=val_dataset, callbacks = [checkpoint])
  
def fit_k_models(k = 5,n_epoch=6, model_name='SE_ResNet',batch_size = 64,large_batch=False,dim=2):
  for i in range(16,k+1):
    if dim == 2:
      strategy, train_dataset, val_dataset = set_batch(batch_size,i)
      if large_batch:
        fit_model(id=i, n_epoch=n_epoch, strategy=strategy, train_dataset=train_dataset, val_dataset=val_dataset, road_weight=False ,model_name=model_name,optimizer_set='lamb')
      else:
        fit_model(id=i, n_epoch=n_epoch, strategy=strategy, train_dataset=train_dataset, val_dataset=val_dataset, road_weight=False ,model_name=model_name)
    else:
      strategy, train_dataset, val_dataset = set_batch(batch_size,i,dim=1)
      if large_batch:
        fit_model(id=i, n_epoch=n_epoch, strategy=strategy, train_dataset=train_dataset, val_dataset=val_dataset, road_weight=False ,model_name=model_name,optimizer_set='lamb')
      else:
        fit_model(id=i, n_epoch=n_epoch, strategy=strategy, train_dataset=train_dataset, val_dataset=val_dataset, road_weight=False ,model_name=model_name)


def load_test_set(dim=2):
  
  GCS_PATH = 'gs://data_bucket_9586/' 
  AUTO = tf.data.experimental.AUTOTUNE # used in tf.data.Dataset API

  if dim == 1:
    def read_test(example):
      features = {'data': tf.io.VarLenFeature(dtype = tf.float32)}
      example = tf.io.parse_single_example(example, features)

      data = tf.sparse.to_dense(example['data'])

      data = tf.reshape(data,(11025,1))
      return data
      
    filenames = tf.io.gfile.glob(GCS_PATH + 'test_wave*.tfrec')
    test_dataset = tf.data.TFRecordDataset(filenames)
    test_dataset = test_dataset.map(read_test).batch(512).prefetch(1)

  else:
    def read_test_2d(example):
      features = {'data': tf.io.VarLenFeature(dtype = tf.float32)}
      example = tf.io.parse_single_example(example, features)

      data = tf.sparse.to_dense(example['data'])

      data = tf.reshape(data,(299,299,1))
      return data
      
    filenames = tf.io.gfile.glob(GCS_PATH + 'test_*.tfrec')[:10]
    test_dataset = tf.data.TFRecordDataset(filenames)
    test_dataset = test_dataset.map(read_test_2d).batch(128).prefetch(1)

  return test_dataset

def make_result(model_num,test_dataset,model_name):
  # 가장 좋은 모델의 weight를 불러옵니다.
  if model_name == 'inception_1':
    model = inception_model_1((299,299,1))
  elif model_name == 'SE_ResNet':
    model = SE_ResNet50((299,299,1),30)
  elif model_name == 'DenseNet':
    model = DenseNet((299,299,1),30)
  elif model_name == '1D_model':
    model = wave_model((11025,1))

  weight_file = glob('drive/My Drive/data/model_{}_{}/*.hdf5'.format(model_name,model_num))[-1]
  print(weight_file)
  model.load_weights(weight_file)
  # 예측 수행
  y_pred = model.predict(test_dataset)

  return y_pred

def k_fold(k, model_name='SE_ResNet',batch_size = 64,n_epoch = 6, train_model=True,large_batch=False,only_train=False,oneDim=False):
  if oneDim:
    test_dataset = load_test_set(dim=1)
  else:   
    #data load
    test_dataset = load_test_set()

  #model_fit
  if train_model:
    if large_batch: 
      if oneDim:
        fit_k_models(k = k,n_epoch = n_epoch, model_name=model_name,batch_size = batch_size,large_batch=True,dim=1)
      else:
        fit_k_models(k = k,n_epoch = n_epoch, model_name=model_name,batch_size = batch_size,large_batch=True)
    else:
      if oneDim:
        fit_k_models(k = k,n_epoch = n_epoch, model_name=model_name,batch_size = batch_size,dim=1)
      else:
        fit_k_models(k = k,n_epoch = n_epoch, model_name=model_name,batch_size = batch_size)

  if not(only_train):
    #make_result
    submission = pd.read_csv('drive/My Drive/data/submission.csv', index_col=0)
    suma = np.zeros_like(submission)
    for i in range(1,k+1):
      suma += make_result(i,test_dataset,model_name=model_name)
    submission.loc[:, :] = suma / k
    submission.to_csv('drive/My Drive/data/submission.csv')
    print('submission.csv is updated!')

# k-folds로 결과 저장

In [ ]:
k_fold(20, model_name='SE_ResNet', batch_size = 64, n_epoch=12, train_model=True)

2
(299, 299, 1)
30


INFO:tensorflow:Initializing the TPU system: grpc://10.81.229.162:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.81.229.162:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Epoch 1/12
   4219/Unknown - 1278s 303ms/step - loss: 1.8262
Epoch 00001: val_loss improved from inf to 1.37677, saving model to drive/My Drive/data/model_SE_ResNet_16/epoch_001_val_1.376767.hdf5
4219/4219 [==============================] - 1310s 310ms/step - loss: 1.8262 - val_loss: 1.3768
Epoch 2/12
4219/4219 [==============================] - ETA: 0s - loss: 1.1384
Epoch 00002: val_loss improved from 1.37677 to 0.92934, saving model to drive/My Drive/data/model_SE_ResNet_16/epoch_002_val_0.929339.hdf5
4219/4219 [==============================] - 661s 157ms/step - loss: 1.1384 - val_loss: 0.9293
Epoch 3/12
4219/4219 [==============================] - ETA: 0s - loss: 0.8552
Epoch 00003: val_loss improved from 0.92934 to 0.68039, saving model to drive/My Drive/data/model_SE_ResNet_16/epoch_003_val_0.680388.hdf5
4219/4219 [==============================] - 656s 156ms/step - loss: 0.8552 - val_loss: 0.6804
Epoch 4/12
4219/4219 [==============================] - ETA: 0s - loss: 0.6654
Epo

In [ ]:
k_fold(15, model_name='SE_ResNet', train_model=False)

drive/My Drive/data/model_SE_ResNet_1/epoch_012_val_0.423293.hdf5
drive/My Drive/data/model_SE_ResNet_2/epoch_012_val_0.391400.hdf5
drive/My Drive/data/model_SE_ResNet_3/epoch_011_val_0.396777.hdf5
drive/My Drive/data/model_SE_ResNet_4/epoch_012_val_0.375129.hdf5
drive/My Drive/data/model_SE_ResNet_5/epoch_012_val_0.369588.hdf5
drive/My Drive/data/model_SE_ResNet_6/epoch_012_val_0.292427.hdf5
drive/My Drive/data/model_SE_ResNet_7/epoch_012_val_0.303550.hdf5
drive/My Drive/data/model_SE_ResNet_8/epoch_012_val_0.221418.hdf5
drive/My Drive/data/model_SE_ResNet_9/epoch_012_val_0.202112.hdf5
drive/My Drive/data/model_SE_ResNet_10/epoch_012_val_0.236830.hdf5
drive/My Drive/data/model_SE_ResNet_11/epoch_012_val_0.251470.hdf5
drive/My Drive/data/model_SE_ResNet_12/epoch_012_val_0.292545.hdf5
drive/My Drive/data/model_SE_ResNet_13/epoch_012_val_0.247642.hdf5
drive/My Drive/data/model_SE_ResNet_14/epoch_010_val_0.264577.hdf5
drive/My Drive/data/model_SE_ResNet_15/epoch_012_val_0.337385.hdf5


# CV-Stacking 

In [ ]:
def make_stacked_data(model_list):
  
  def load_data(seed=1,dim=2):
    GCS_PATH = 'gs://data_bucket_9586/' 
    AUTO = tf.data.experimental.AUTOTUNE # used in tf.data.Dataset API
    if dim == 2:
      def read_data(example):
        features = {'data': tf.io.VarLenFeature(dtype = tf.float32)}
        example = tf.io.parse_single_example(example, features)

        data = tf.sparse.to_dense(example['data'])

        data = tf.reshape(data,(299,299,1))
        return data
        
      def read_label(example):
        features = {'label': tf.io.VarLenFeature(dtype = tf.float32)}
        example = tf.io.parse_single_example(example, features)

        data = tf.sparse.to_dense(example['label'])

        data = tf.reshape(data,(30,))
        return data

      filenames = tf.io.gfile.glob(GCS_PATH + 'train_resize_*.tfrec')[:100]
      np.random.seed(seed)
      np.random.shuffle(filenames)
      filenames = filenames[:10]
    
    else:
      def read_data(example):
        features = {'data': tf.io.VarLenFeature(dtype = tf.float32)}
        example = tf.io.parse_single_example(example, features)

        data = tf.sparse.to_dense(example['data'])

        data = tf.reshape(data,(11025,1))
        return data
        
      def read_label(example):
        features = {'label': tf.io.VarLenFeature(dtype = tf.float32)}
        example = tf.io.parse_single_example(example, features)

        data = tf.sparse.to_dense(example['label'])

        data = tf.reshape(data,(30,))
        return data

      filenames = tf.io.gfile.glob(GCS_PATH + 'train_wave_*.tfrec')[:100]
      np.random.seed(seed)
      np.random.shuffle(filenames)


    data = tf.data.TFRecordDataset(filenames)
    data = data.map(read_data).batch(1024).prefetch(1)

    label = tf.data.TFRecordDataset(filenames)
    label = label.map(read_label)
      
    return data,label

  data,label = load_data()
  test_dataset = load_test_set()  

  data_1d,label_1d = load_data(dim=1)
  test_dataset_1d = load_test_set(dim=1)
    
  print('loading is complete!')
  predict = np.array([])
  for j in model_list:
    for i in range(8,j[1]+1):    
      #일부에 대한 출력을 만듬
      if j[0] == '1D_model':
        new = make_result(i, data_1d,  j[0])
        new_test = make_result(i, test_dataset_1d, j[0])
      else:
        new = make_result(i, data,  j[0])
        new_test = make_result(i, test_dataset, j[0])
      print('prediction by {}_m{} is done'.format(j[0],i))
      if predict.any() :
        #해당 결과들 stacking
        predict = np.concatenate((predict,new), axis=1)
        predict_test =  np.concatenate((predict_test,new_test), axis=1)
      else:
        predict = new
        predict_test = new_test

  print('predict is done!')

  pd.DataFrame(predict).to_pickle('drive/My Drive/data/stacked_x')
  pd.DataFrame(predict_test).to_pickle('drive/My Drive/data/stacked_x_test')

if __name__ == '__main__':
  model_list = [('SE_ResNet',10)]
  make_stacked_data(model_list)

loading is complete!
drive/My Drive/data/model_SE_ResNet_8/epoch_012_val_0.221418.hdf5
drive/My Drive/data/model_SE_ResNet_8/epoch_012_val_0.221418.hdf5
prediction by SE_ResNet_m8 is done
drive/My Drive/data/model_SE_ResNet_9/epoch_012_val_0.202112.hdf5
drive/My Drive/data/model_SE_ResNet_9/epoch_012_val_0.202112.hdf5
prediction by SE_ResNet_m9 is done
drive/My Drive/data/model_SE_ResNet_10/epoch_012_val_0.236830.hdf5
drive/My Drive/data/model_SE_ResNet_10/epoch_012_val_0.236830.hdf5
prediction by SE_ResNet_m10 is done
predict is done!


In [ ]:
def cv_stacking(nets=5,total_models=5):
  #데이터 로드
  X = pd.read_pickle('drive/My Drive/data/stacked_x').values
  predict_test = pd.read_pickle('drive/My Drive/data/stacked_x_test').values
  label = pd.read_csv('drive/My Drive/data/stacked_y.csv',index_col=0).values
  #test data
  from sklearn.model_selection import train_test_split
  X, X_test, label, label_test = train_test_split(X, label,test_size=0.1, shuffle=False,random_state=1004)


  #신경망 학습
  submission = pd.read_csv('drive/My Drive/data/submission.csv', index_col=0)
  suma = np.zeros_like(submission)
  test = np.zeros_like(label_test, dtype='float64')
  nodes = total_models*30

  for t in range(nets):
    np.random.seed(t)
    new_X = np.random.permutation(X) 
    np.random.seed(t)
    new_label = np.random.permutation(label)

    spit_len = int(len(new_X) * 0.9)   
    X_train = new_X[:spit_len]
    X_val = new_X[spit_len:]

    label_train = new_label[:spit_len]
    label_val = new_label[spit_len:]

    def simple_net(input_shape):
      inp = Input(input_shape)
      out = Dense(30,activation='softmax')(inp)
      return Model(inp,out)

    model = simple_net((nodes,))
    model.compile(loss=tf.keras.losses.KLDivergence(), optimizer = 'adam')
    model.fit(X_train, label_train,epochs=12, batch_size=8,validation_data=(X_val, label_val))

    suma += model.predict(predict_test)
    test += model.predict(X_test)

    

    print(str((t+1)) + ' of ' + str(nets) + ' is done')

  
  #테스트
  kl = tf.keras.losses.KLDivergence()
  score = kl(label_test, test/nets).numpy()
  print('score is '+ str(score))
  
  
  #결과 만들기
  submission.loc[:, :] = suma / nets
  submission.to_csv('drive/My Drive/data/submission_stack.csv')
  print('submission_stack.csv is updated!')

# 8/4
--------------------------------------

**모델** : SE_ResNet50(5-folds,6 epochs)
  
**데이터**: x_trian_mel-spectogram(299*299)
  
**score** : **0.72265 (15위)**

**메모** : shifted aug not good, k-folds good, 에폭 더 주면 많이 좋을 듯..

**피드백** : 12에폭으로 결과 확인

--------------------------------------

# 8/5
--------------------------------------

**모델** : SE_ResNet50(5-folds,12 epochs)
  
**데이터**: x_trian_mel-spectogram(299*299)
  
**score** : **0.71xx (16위)**

**메모** : 0.7대에서 성능 개선 없음, 드랍아웃 적용하고 에폭더 주면 좋을듯 

**피드백** : 드랍아웃 추가 3-fold, 20epoch

--------------------------------------



# 8/11
--------------------------------------

**모델** : SE_ResNet50(10-folds,12 epochs)
  
**데이터**: x_trian_mel-spectogram(299*299)
  
**score** : **0.62657 (13위)**

**메모** : 드랍아웃 적용하고 5개 추가해서 대폭성능 개선

**피드백** : 5fold 추가, L2 model 적용해보기..

--------------------------------------

--------------------------------------

**모델** : SE_ResNet50(15-folds,12 epochs)
  
**데이터**: x_trian_mel-spectogram(299*299)
  
**score** : **0.6037 (13위)**

**메모** : 성능 소폭 개선됨, epoch 늘릴시 빠르게 수렴하나 성능 저하가 의심됨..

**피드백** : 20 fold로 결과 확인

--------------------------------------